# Tweet Language Classification

A common text classification task involves automatically determining the language in which a document is written, based on previously-labelled example documents.

In this notebook, we will look at automatically classifying the text from tweets as either English or non-English. The dataset we will use is a subset of the [UMass Global English on Twitter Dataset](https://www.kaggle.com/rtatman/the-umass-global-english-on-twitter-dataset).

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, RepeatedKFold
%matplotlib inline

## Data Preprocessing

Firstly, read the tab-separated dataset file into a Pandas Data Frame. Here each tweet will be considered as a document. Note that URLs and user mentions have already been removed from this data.

In [ ]:
df = pd.read_csv("tweet-language.tsv", sep="\t").set_index("Tweet ID")
print("Read %d documents" % len(df))

In [ ]:
df.head(10)

Our documents here are given by the *Tweet* column. Our target variable is given by the *English* column, where 1 indicates an English language tweet and 0 indicates a non-English tweet.

In [ ]:
# 0 = Non-English, 1 = English
target_categories = ["Non-English", "English"]
# Content for all documents
documents = df["Tweet"]
# Class labels for all documents
target = df["English"]

We can check to see how balanced the classes in the dataset are, based on the values in the target:

In [ ]:
target.value_counts()

Next, we will produce vector representations of the documents. For real applications we would want to use a custom tokenizer to handle the specifics of tweets (e.g. mentions, hashtags etc). However, for this simple example we will just use the standard scikit-learn tokenizer and a simple *CountVectorizer*.

Note that we do not use any "stop words" here. For language detection, common stop words might actually prove to be useful features.

In [ ]:
vectorizer = CountVectorizer(min_df = 10, stop_words=None)
X = vectorizer.fit_transform(documents)
print(X.shape)

In [ ]:
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

In [ ]:
# display a set of sample terms
print(terms[150:170])

## Classification and Evaluation - Approach 1

Firstly, we will look at building a k-NN model, and evaluating its performance using a single training/test split.

Split the data, so that we have 70% in the training set and 30% in the test set:

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(X, target, test_size=0.3)

In [ ]:
print("Training set has %d examples" % data_train.shape[0] )
print("Test set has %d examples" % data_test.shape[0] )

Prepare the k-NN classification model, for 3 nearest neighbours in this case:

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(data_train, target_train)

Make predictions for the tweets in the test set. Remember that 1 indicates an English languages tweet and 0 indicates a non-English tweet.

In [ ]:
predicted = model.predict(data_test)
predicted

We could look at a few examples where the classifier made a mistakee:

In [ ]:
# examples of non-English tweets being predicted as English
num_examples = 0 
for i, predicted_class in enumerate(predicted):
    if target_test.iloc[i] == 0 and predicted_class == 1:
        tweet_id = target_test.index[i]
        print( "%s: %s" % ( tweet_id, df.loc[tweet_id]["Tweet"] ) )
        num_examples += 1
    if num_examples == 5:
        break

In [ ]:
# examples of English tweets being predicted as non-English
num_examples = 0 
for i, predicted_class in enumerate(predicted):
    if target_test.iloc[i] == 1 and predicted_class == 0:
        tweet_id = target_test.index[i]
        print( "%s: %s" % ( tweet_id, df.loc[tweet_id]["Tweet"] ) )
        num_examples += 1
    if num_examples == 5:
        break

Now we will evaluate the performance of the classifier. Firstly, we can just look at the overall accuracy.

In [ ]:
print("Accuracy = %.4f" % accuracy_score(target_test, predicted))

Next, we will look at the confusion matrix to see where the errors lie:

In [ ]:
# calculate the matrix 
cm = confusion_matrix(target_test, predicted, labels=[0,1])
# display it graphically
cmd = ConfusionMatrixDisplay(cm, display_labels=target_categories)
cmd.plot();

We can look at *precision* and *recall* scores for both classes.

In [ ]:
print("Precision (English) = %.4f" % precision_score(target_test, predicted, pos_label=1) )
print("Recall (English) = %.4f" % recall_score(target_test, predicted, pos_label=1) )

In [ ]:
print("Precision (Non-English) = %.4f" % precision_score(target_test, predicted, pos_label=0) )
print("Recall (Non-English) = %.4f" % recall_score(target_test, predicted, pos_label=0) )

We can compute a single value to summarise the performance on each class, using the F1-measure:

In [ ]:
print("F1 (English) = %.4f" % f1_score(target_test, predicted, pos_label=1) )
print("F1 (Non-English) = %.4f" % f1_score(target_test, predicted, pos_label=0) )

The *classification_report()* functions provides a convenient way of summarising all of this information. Note that *support* here indicates the number of actual occurrences of a class in the test data.

In [ ]:
print(classification_report(target_test, predicted, target_names=target_categories))

## Classification and Evaluation - Approach 2

If we re-run the evaluation above several times, we will get different performance scores depending on the randomly-generated training/test split that we are using. A more robust strategy involves using *k-fold cross-validation* to evaluate a classifier.

In the example below, we will evaluate a k-NN classifier using 5-fold cross validation. The model in each fold will be evaluated using accuracy.

In [ ]:
# create a single classifier
model = KNeighborsClassifier(n_neighbors=3)
# apply 5-fold cross-validation, measuring accuracy each time
acc_scores = cross_val_score(model, X, target, cv=5, scoring="accuracy")

In [ ]:
# represent the results as a Pandas Series
labels = ["Fold %d" % i for i in range(1,len(acc_scores)+1)]
s_acc = pd.Series(acc_scores, index = labels)
s_acc

In [ ]:
print("Mean accuracy: %.4f" % s_acc.mean())

If we wanted to be very thorough, we could repeat this process multiple times (e.g. 10 runs, each with 5 folds). This might take a while, depending on the size of the dataset.

In [ ]:
rkf = RepeatedKFold(n_splits=5, n_repeats=10)
# note that we pass RepeatedKFold as the parameter cv
repeated_acc_scores = cross_val_score(model, X, target, cv=rkf, scoring="accuracy")

In [ ]:
s_racc = pd.Series(repeated_acc_scores)
s_racc.head(20)

Calculate the mean over all values in the Data Frame:

In [ ]:
print("Overall mean accuracy: %.4f" % s_racc.mean() )
print("Overall standard deviation in accuracy: %.4f" % s_racc.std() )

We could use this evaluation process to compare the performance of different classification algorithms on a given corpus.

## Classification and Evaluation with Pipelines

As we saw in the last lab notebook, we can the Scikit-learn *Pipeline* class to connect together a sequence of steps for classification. We can use this to process training and test sets separately when performing evaluations of classifications.

Firstly we define the steps in our pipeline:

In [ ]:
from sklearn.pipeline import Pipeline
pipeline1 = Pipeline([
    ('vec', CountVectorizer(min_df = 10, stop_words=None)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier(n_neighbors=3))
])

Then apply *k-fold cross-validation* using the pipeline for each fold. Note that we pass in the raw documents, as all of the preprocessing steps will be performing for us as part of the pipeline.

In [ ]:
# apply 5-fold cross-validation, measuring accuracy each time
acc_scores = cross_val_score(pipeline1, documents, target, cv=5, scoring="accuracy")
s_acc = pd.Series(acc_scores)
print("Mean accuracy: %.4f" % s_acc.mean())

Now change the process to use a linear Support Vector Machine (SVM) classifier, and perform the evaluation again:

In [ ]:
pipeline2 = Pipeline([
    ('vec', CountVectorizer(stop_words="english")),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())
])

In [ ]:
# apply 5-fold cross-validation, measuring accuracy each time
acc_scores = cross_val_score(pipeline2, documents, target, cv=5, scoring="accuracy")
s_acc = pd.Series(acc_scores)
print("Mean accuracy: %.4f" % s_acc.mean())